In [1]:
!pip install -U notebook-as-pdf

In [6]:
#!pyppeteer-install
!pip install pyppeteer

In [ ]:
!pip uninstall pandoc

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as mpl
import sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc

Exploratory Data Analysis

In [ ]:
loantap=pd.read_csv(r'C:\Users\Narsingha.Addepalli\OneDrive - Neudesic\Learning\scaler\businesscase\logistic_regression.csv')

In [ ]:
loantap.info()

In [ ]:
loantap.describe()

In [ ]:
def simpleeda1(inputdf):
    for x in inputdf.columns:
        print(x,len(inputdf[x]),type(inputdf[x]),type(inputdf[x][0]))
        print(inputdf[x].value_counts())
simpleeda1(loantap)

In [ ]:
#Simple EDA to check distribution of integer columns
def simpleeda(inputdf):
    for x in inputdf.columns:
        print(x,len(inputdf[x]),type(inputdf[x]),type(inputdf[x][0]))
#         if type(inputdf[x][0])=='int' or type(inputdf[x][0])=='float64':
        if inputdf.dtypes[x]=='int':
            mpl.hist(inputdf[x],bins=10,histtype='bar')
            mpl.show()
        elif inputdf.dtypes[x]=='float':
            mpl.plot(inputdf[x])
            mpl.show()
        else:
            print("column is of type",inputdf.dtypes[x])
#             mpl.title("histogram")
#             mpl.xlabel("bin number")
#             mpl.ylabel("values")
#             mpl.show()
simpleeda(loantap)

Several outliers show up in the above Charts

So Lets do some data cleansing

In [ ]:
loantap_clean=loantap[loantap['dti']<1000.0]

In [ ]:
mpl.hist(loantap_clean['dti'],bins=10,histtype='bar')
mpl.show()

Find correlation

In [ ]:
correlation_matrix = loantap_clean.corr()
print(correlation_matrix)

Feature Engineering

In [ ]:
loantap_clean.loc[loantap_clean['pub_rec'] >1.0, 'pub_rec'] =1

In [ ]:
loantap_clean.loc[loantap_clean['mort_acc'] >1.0, 'mort_acc'] =1

In [ ]:
loantap_clean.loc[loantap_clean['pub_rec_bankruptcies'] >1.0, 'pub_rec_bankruptcies'] =1

Handle missing values

In [ ]:
# Handle missing values (assuming fill with median for demonstration)
#loantap_clean.fillna(0, inplace=True)

#Simple EDA to check distribution of integer columns
def replacemissing(inputdf):
    for x in inputdf.columns:
        print(x,len(inputdf[x]),type(inputdf[x]),type(inputdf[x][0]))
        if type(inputdf[x][0])=='int' or type(inputdf[x][0])=='float64':
            input_df.loc[inputdf[x]==NaN,x] =1
        else:
            print("column is of type",inputdf.dtypes[x])
#             mpl.title("histogram")
#             mpl.xlabel("bin number")
#             mpl.ylabel("values")
#             mpl.show()
replacemissing(loantap_clean)

Split the model into train and test datasets

In [ ]:
# Split the data into training and testing sets
#X_train, X_test, y_train, y_test = train_test_split(loantap_clean['loan_amnt','int_rate','installment','annual_inc','dti','open_acc','pub_rec','revol_bal','revol_util','total_acc','mort_acc','pub_rec_bankruptcies', loantap_clean['loan_status'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(loantap_clean['loan_amnt','int_rate','installment','annual_inc','dti','open_acc','pub_rec','revol_bal','revol_util','total_acc','mort_acc','pub_rec_bankruptcies'], loantap_clean['loan_status'], test_size=0.2, random_state=42)

In [ ]:
# Standardize continuous variables
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
x_test=scaler.transform(X_train)

In [ ]:
# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# evaluate the model
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [ ]:
# Visualize the decision boundary with accuracy information
plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_test[:, 2], y=X_test[:, 8], hue=y_test, palette={0: 'blue', 1: 'red'}, marker='o')
plt.xlabel("loan_amnt")
plt.ylabel("loan_status")
plt.title("Logistic Regression Decision Boundary\nAccuracy: {:.2f}%".format(accuracy * 100))
plt.legend(title="loantap", loc="upper right")
plt.show()


In [ ]:
# Plot ROC Curve
y_prob = model.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2,
		label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve\nAccuracy: {:.2f}%'.format(
	accuracy * 100))
plt.legend(loc="lower right")
plt.show()